In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import math

In [2]:
def fuzzify_small(x, r_max):
    d = r_max / 2
    c = d / 3
    return tf.clip_by_value((d - x) / (d - c), 0, 1)

def fuzzify_medium(x, r_max):
    a = r_max / 4
    m = r_max / 2
    b = m + a
    left = tf.clip_by_value((x - a) / (m - a), 0, 1)
    right = tf.clip_by_value((b - x) / (b - m), 0, 1)
    return tf.minimum(left, right)

def fuzzify_large(x, r_max):
    r = r_max / 2
    q = r + r_max / 4
    return tf.clip_by_value((x - r) / (q - r), 0, 1)

def fuzzy_pooling(inputs, pool_size=(2, 2), strides=None, padding='VALID'):
    if strides is None:
        strides = pool_size

    r_max = 6.0

    # Fuzzification
    small = fuzzify_small(inputs, r_max)
    medium = fuzzify_medium(inputs, r_max)
    large = fuzzify_large(inputs, r_max)

    # Fuzzy aggregation
    fuzzy_sets = tf.stack([small, medium, large], axis=-1)

    # Compute scores for each fuzzy set
    scores = tf.reduce_mean(fuzzy_sets, axis=[1, 2, 3])
    selected = tf.argmax(scores, axis=-1)

    # Prepare indices for defuzzification
    batch_size = tf.shape(inputs)[0]
    height = tf.shape(inputs)[1]
    width = tf.shape(inputs)[2]
    channels = tf.shape(inputs)[3]

    selected = tf.cast(selected, tf.int32)
    selected = tf.reshape(selected, [batch_size, 1, 1, 1])
    selected = tf.tile(selected, [1, height, width, channels])

    indices = tf.stack([
        tf.broadcast_to(tf.range(batch_size)[:, None, None, None], [batch_size, height, width, channels]),
        tf.broadcast_to(tf.range(height)[None, :, None, None], [batch_size, height, width, channels]),
        tf.broadcast_to(tf.range(width)[None, None, :, None], [batch_size, height, width, channels]),
        tf.broadcast_to(tf.range(channels)[None, None, None, :], [batch_size, height, width, channels]),
        selected
    ], axis=-1)

    selected_sets = tf.gather_nd(fuzzy_sets, indices)

    # Defuzzification
    numerator = tf.nn.avg_pool2d(inputs * selected_sets, pool_size, strides, padding)
    denominator = tf.nn.avg_pool2d(selected_sets, pool_size, strides, padding)

    return numerator / (denominator + 1e-10)

In [3]:
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 5us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 15s 1us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 3s 1us/step


In [4]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = tf.keras.layers.Lambda(lambda x: fuzzy_pooling(x, pool_size=(2, 2), strides=(2, 2)))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.Lambda(lambda x: fuzzy_pooling(x, pool_size=(2, 2), strides=(2, 2)))(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [5]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_1 (Lambda)                    │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         102,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 121,930 (476.29 KB)

 Trainable params: 121,930 (476.29 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(x_train, y_train,
                   epochs=10,
                   validation_data=(x_test, y_test),
                   batch_size=32)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 166s 87ms/step - accuracy: 0.7346 - loss: 0.7243 - val_accuracy: 0.8516 - val_loss: 0.4271
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 162s 86ms/step - accuracy: 0.8652 - loss: 0.3716 - val_accuracy: 0.8688 - val_loss: 0.3608
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 165s 88ms/step - accuracy: 0.8864 - loss: 0.3125 - val_accuracy: 0.8782 - val_loss: 0.3367
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 167s 89ms/step - accuracy: 0.8968 - loss: 0.2769 - val_accuracy: 0.8848 - val_loss: 0.3104
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 167s 89ms/step - accuracy: 0.9049 - loss: 0.2540 - val_accuracy: 0.8952 - val_loss: 0.2913
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 166s 88ms/step - accuracy: 0.9133 - loss: 0.2337 - val_accuracy: 0.8985 - val_loss: 0.2678
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 167s 89ms/step - accuracy: 0.9191 - loss: 0.2161 - val_accuracy: 0.9003 - val_loss: 0.2766
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 167s 89ms/step - accuracy: 

In [7]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 8s - 27ms/step - accuracy: 0.9074 - loss: 0.2585

Test accuracy: 0.9074000120162964


In [8]:
inputs = tf.keras.Input(shape=(28, 28, 1))

# First convolutional block
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.25)(x)

# Second convolutional block
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.25)(x)

# Third convolutional block
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.25)(x)

# Dense layers
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model_1 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [9]:
model_1.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model_1.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 14, 14, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 14, 14, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 14, 14, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 7, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 7, 7, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 7, 7, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 3, 3, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 3, 3, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 3, 3, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 2,463,178 (9.40 MB)

 Trainable params: 2,460,746 (9.39 MB)

 Non-trainable params: 2,432 (9.50 KB)

In [10]:
history = model_1.fit(x_train, y_train,
                   epochs=10,
                   validation_data=(x_test, y_test),
                   batch_size=32)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 244s 126ms/step - accuracy: 0.7060 - loss: 0.8888 - val_accuracy: 0.8797 - val_loss: 0.3414
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 238s 127ms/step - accuracy: 0.8726 - loss: 0.3535 - val_accuracy: 0.8709 - val_loss: 0.3402
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 239s 127ms/step - accuracy: 0.8935 - loss: 0.3014 - val_accuracy: 0.9074 - val_loss: 0.2524
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 247s 132ms/step - accuracy: 0.9071 - loss: 0.2643 - val_accuracy: 0.9149 - val_loss: 0.2341
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 258s 138ms/step - accuracy: 0.9142 - loss: 0.2390 - val_accuracy: 0.9118 - val_loss: 0.2402
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 244s 130ms/step - accuracy: 0.9232 - loss: 0.2182 - val_accuracy: 0.9256 - val_loss: 0.2103
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 249s 133ms/step - accuracy: 0.9258 - loss: 0.2044 - val_accuracy: 0.9234 - val_loss: 0.2133
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 254s 135ms/step - ac

In [11]:
test_loss, test_acc = model_1.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 13s - 42ms/step - accuracy: 0.9352 - loss: 0.1888

Test accuracy: 0.9351999759674072
